In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import matplotlib.dates as mdates
import os, sys
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm
import pandas as pd


def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

r1 = MakeDataFromText("Temperature 50p5/RSSI_66212.csv")
t1 = MakeDataFromText("Temperature 50p5/tags_66212.csv")
r2 = MakeDataFromText("Temperature 50p8/RSSI_95451.csv")
t2 = MakeDataFromText("Temperature 50p8/tags_95451.csv")
r3 = MakeDataFromText("40p7/RSSI_56307.csv")
t3 = MakeDataFromText("40p7/tags_56307.csv")
r4 = MakeDataFromText("56p2 Data 1/RSSI_44986.csv")
t4 = MakeDataFromText("56p2 Data 1/tags_44986.csv")
r5 = MakeDataFromText("56p2 Data 2/RSSI_44926.csv")
t5 = MakeDataFromText("56p2 Data 2/tags_44926.csv")
t6 = MakeDataFromText("55p5/tags_69303.csv")
r6 = MakeDataFromText("55p5/RSSI_69303.csv")
t7 = MakeDataFromText("23p7/tags_13318.csv")
r7 = MakeDataFromText("23p7/RSSI_13318.csv")
t8 = MakeDataFromText("23p2/tags_60650.csv")
r8 = MakeDataFromText("23p2/RSSI_60650.csv")
r9 = MakeDataFromText("33p3/RSSI_96210.csv")
t9 = MakeDataFromText("33p3/tags_96210.csv")
t20 = MakeDataFromText("54/tags_25812.csv")
r20 = MakeDataFromText("54/RSSI_25812.csv")
t21 = MakeDataFromText("54p2/tags_55133.csv")
r21 = MakeDataFromText("54p2/RSSI_55133.csv")
t22 = MakeDataFromText("54p2/tags_88633.csv")
r22 = MakeDataFromText("54p2/RSSI_88633.csv")
t10 = MakeDataFromText("1527 23p2/tags_41059.csv")
r10 = MakeDataFromText("1527 23p2/RSSI_41059.csv")

temp23p2 = MakeDict([t8])
rssi23p2 = MakeDict([r8])
temp50p5 = MakeDict([t1])
rssi50p5 = MakeDict([r1])
temp50p8 = MakeDict([t2])
rssi50p8 = MakeDict([r2])
temp40p7 = MakeDict([t3])
rssi40p7 = MakeDict([r3])
temp56p2_1 = MakeDict([t4])
rssi56p2_1 = MakeDict([r4])
temp56p2_2 = MakeDict([t5])
rssi56p2_2 = MakeDict([r5])
temp55p5 = MakeDict([t6])
rssi55p5 = MakeDict([r6])
temp23p7 = MakeDict([t7])
rssi23p7 = MakeDict([r7])
temp33p3 = MakeDict([t9])
rssi33p3 = MakeDict([r9])
temp54 = MakeDict([t20])
rssi54 = MakeDict([r20])
temp54p2_1 = MakeDict([t21])
rssi54p2_1 = MakeDict([r21])
temp54p2_2 = MakeDict([t22])
rssi54p2_2 = MakeDict([r22])
temp23p2_1527 = MakeDict([t10])
rssi23p2_1527 = MakeDict([r10])

In [ ]:
LABEL = [ "23.2", "23.7", "33.3", "40.7", "50.5", "50.8","54.2 #1", "54.2 #2" "55.5", ]
TEMP = [ temp23p2, temp23p2_1527, temp23p7, temp33p3, temp40p7, temp50p5, temp50p8, temp54, temp54p2_1, temp54p2_2, temp55p5, ]
RSSI = [ rssi23p2, rssi23p2_1527, rssi23p7, rssi33p3, rssi40p7, rssi50p5, rssi50p8, rssi54, rssi54p2_1, rssi54p2_2, rssi55p5, ]
FITCOLOR = [ "#ff0000", "#ff6f00", "#ff14d0", "#fa0505", "#f07e05", "#ff14d0", "#ff14d0", "#fa0505", "#ff14d0" ]
RAWCOLOR = [ "#0DA995", "#0024c2", "#0F78BC", "#008ec2", "#110063", "#0024c2", "#008ec2", "#110063", "#0024c2" ]
TAGS = [ '2B42', '3943', '3242' ]

In [ ]:
with pd.ExcelWriter("_tags_coefficients_.xlsx") as writer:
    for SHEETS in range(3):
        list_dfs = []
        for TAG in TAGS:
            r2_l, x2_l, x_l, c_l = [], [], [], []
            r2_q, x2_q, x_q, c_q = [], [], [], []
            for u in range( 11 ):
                for k,v in TEMP[u].items():
                    if k[-4:]==TAG:
                        st, sr = sorted( TEMP[u][k].items() ), sorted( RSSI[u][k].items() )
                        dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

                        # Making Standard Deviation and Error Bars
                        means, rssi = [], []
                        c1 = { i:[] for i in range(26) }
                        for i in range(len(dt)): c1[dr[i]].append(dt[i])
                        for k,v in c1.items():
                            if SHEETS==0:
                                if len(v)>0 and k<21:
                                    mean = sum(v)/len(v)
                                    std = np.std(v)
                                    rssi.append(k)
                                    means.append(mean)
                            if SHEETS==1:
                                if len(v)>0 and k>20:
                                    mean = sum(v)/len(v)
                                    std = np.std(v)
                                    rssi.append(k)
                                    means.append(mean)
                            if SHEETS==2:
                                if len(v)>0:
                                    mean = sum(v)/len(v)
                                    std = np.std(v)
                                    rssi.append(k)
                                    means.append(mean)
                        try:
                            # Making Best Fits, Linear & Quadratic
                            model1 = np.poly1d( np.polyfit(rssi, means,  1) )
                            model2 = np.poly1d( np.polyfit(rssi, means,  2) )

                            x1_, y1 = zip( *sorted( zip(rssi, model1(rssi)) ) )
                            x2_, y2 = zip( *sorted( zip(rssi, model2(rssi)) ) )

                            strmodel1 = ", " + str(round(model1[1], 4))+"x + "+str(round(model1[0], 4))
                            strmodel2 = ", " + str(round(model2[2], 4))+"x^2 + "+str(round(model2[1], 4))+"x + "+str(round(model2[0], 4))

                            r2_l.append( round(r2_score(means, y1), 3) )
                            r2_q.append( round(r2_score(means, y2), 3) )
                            x2_l.append(np.nan)
                            x2_q.append( round(model2[2], 4) )
                            x_l.append( round(model1[1], 4) )
                            x_q.append( round(model2[1], 4) )
                            c_l.append( round(model1[0], 4) )
                            c_q.append( round(model2[0], 4) )
                        except:
                            r2_l.append(np.nan)
                            r2_q.append(np.nan)
                            x2_l.append(np.nan)
                            x2_q.append(np.nan)
                            x_l.append(np.nan)
                            x_q.append(np.nan)
                            c_l.append(np.nan)
                            c_q.append(np.nan)
                        
            index = [
                '23.2 4/27 18:28 Linear',
                '23.2 5/2  15:27 Linear',
                '23.7 Linear',
                "33.3 Linear",
                '40.7 Linear',
                '50.5 Linear',
                '50.8 Linear',
                '54.0 Linear',
                '54.2 #1 Linear', 
                '54.2 #2 Linear',
                '55.5 Linear', 
                'Linear Avg, Std',
                '23.2 4/27 18:28 Quadratic',
                '23.2 5/2  15:27 Quadratic',
                '23.7 Quadratic',
                "33.3 Quadratic",
                '40.7 Quadratic',
                '50.5 Quadratic',
                '50.8 Quadratic',
                '54.0 Quadratic',
                '54.2 #1 Quadratic',
                '54.2 #2 Quadratic',
                '55.5 Quadratic',
                'Quadratic Avg, Std'
            ]

            r2_l.append('n/a')
            x2_l.append( str(round(np.mean(x2_l),3)) + ", " + str(round(np.std(x2_l),6))  )
            x_l.append( str(round(np.mean(x_l),3)) + ", " + str(round(np.std(x_l),6)) )
            c_l.append('n/a')

            r2_q.append('n/a')
            x2_q.append( str(round(np.mean(x2_q),3)) + ", " + str(round(np.std(x2_q),6)) )
            x_q.append( str(round(np.mean(x_q),3)) + ", " + str(round(np.std(x_q),6)) )
            c_q.append('n/a')

            print(len(index))
            print(len(r2_l+r2_q))
            print(len(x2_l+x2_q))
            print(len(x_l+x_q))
            print(len(c_l+c_q))

            smalldf = pd.DataFrame(
                data={ 
                    'index': index,
                    TAG + ', R2': r2_l + r2_q,
                    TAG + ', x2': x2_l + x2_q,
                    TAG + ', x': x_l + x_q,
                    TAG + ', c': c_l + c_q
                },
                columns=['index', TAG + ', R2', TAG + ', x2', TAG + ', x', TAG + ', c']
            )

            smalldf.set_index('index')
            list_dfs.append(smalldf)


        df = list_dfs[0]
        for t__ in list_dfs[1:]:
            df = pd.merge(df, t__, on='index')
        df = df.set_index('index')

        # if SHEETS==0:
        #     df.to_excel(writer, sheet_name='RSSI<21')
        # if SHEETS==1:
        #     df.to_excel(writer, sheet_name='RSSI>20')
        # if SHEETS==2:
        #     df.to_excel(writer, sheet_name='All RSSI')